In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import LDA
import numpy as np

In [ ]:
from pyspark.mllib.classification import LogisticRegressionModel,LogisticRegressionWithLBFGS, SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.clustering import *

In [ ]:
path = "/content/drive/MyDrive/data/lab3_datafile/lab3_data.txt"

In [ ]:
df = spark.read.format("libsvm").load(path)

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(4,[0,1,2,3],[5.1...|
|  0.0|(4,[0,1,2,3],[4.9...|
|  0.0|(4,[0,1,2,3],[4.7...|
|  0.0|(4,[0,1,2,3],[4.6...|
|  0.0|(4,[0,1,2,3],[5.0...|
|  0.0|(4,[0,1,2,3],[5.4...|
|  0.0|(4,[0,1,2,3],[4.6...|
|  0.0|(4,[0,1,2,3],[5.0...|
|  0.0|(4,[0,1,2,3],[4.4...|
|  0.0|(4,[0,1,2,3],[4.9...|
|  0.0|(4,[0,1,2,3],[5.4...|
|  0.0|(4,[0,1,2,3],[4.8...|
|  0.0|(4,[0,1,2,3],[4.8...|
|  0.0|(4,[0,1,2,3],[4.3...|
|  0.0|(4,[0,1,2,3],[5.8...|
|  0.0|(4,[0,1,2,3],[5.7...|
|  0.0|(4,[0,1,2,3],[5.4...|
|  0.0|(4,[0,1,2,3],[5.1...|
|  0.0|(4,[0,1,2,3],[5.7...|
|  0.0|(4,[0,1,2,3],[5.1...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
df.describe("label").show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               150|
|   mean|               1.0|
| stddev|0.8192319205190406|
|    min|               0.0|
|    max|               2.0|
+-------+------------------+



In [ ]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', labelCol='label')

In [ ]:
model=logr.fit(df)

In [ ]:
model_prediction = model.transform(df)

In [ ]:
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
weightedPrecision = evaluatorMulti.evaluate(model_prediction, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(model_prediction, {evaluatorMulti.metricName: "weightedRecall"})
Accuracy = evaluatorMulti.evaluate(model_prediction, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(model_prediction, {evaluatorMulti.metricName: "f1"})

In [ ]:
weightedPrecision

0.9866666666666666

In [ ]:
weightedRecall

0.9866666666666666

In [ ]:
Accuracy

0.9866666666666667

In [ ]:
f1

0.9866666666666666